In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from scipy import sparse
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [4]:
y = pd.read_excel('F:/Subtraction/price/XXX20220628.xlsx', sheet_name=0).fillna(-1)

In [5]:
# usecols=['（配）所属区间', '（工）所属区间']
Y = np.vstack((y['（配）所属区间'].astype('str'), y['（工）所属区间'].astype('str'))).T
Y

array([['A', '-1'],
       ['A', 'A'],
       ['A', 'A'],
       ...,
       ['A', 'A'],
       ['A', '-1'],
       ['A', 'C']], dtype=object)

In [6]:
y

,registno,frameno,车型名称,修理厂类型,licenseno,compname,配件定损,配件核损,工时定损小计,工时核损小计,工时名称,配件减损,工时减损,hourComname,（配）max-min÷X,（配）所属区间,（工）max-min÷X,（工）所属区间
0,RDAA2021510100S0256472,LSGDR53L8MH112602,2021 上汽通用凯迪拉克 凯迪拉克CT4 三厢 2.0T 手自一体 尊贵型(SGM7202...,1,川Z2DG97,前叶子板内衬（右）,125.00,124.34,-1.00,-1.00,无,未减,未减,-1,0,A,,-1
1,RDAA2021510100S0256472,LSGDR53L8MH112602,2021 上汽通用凯迪拉克 凯迪拉克CT4 三厢 2.0T 手自一体 尊贵型(SGM7202...,1,川Z2DG97,前大灯（右）,6602.00,4774.38,34.20,34.20,拆装,未减,减损,前大灯（右）,0,A,0,A
2,RDAA2021510100S0256472,LSGDR53L8MH112602,2021 上汽通用凯迪拉克 凯迪拉克CT4 三厢 2.0T 手自一体 尊贵型(SGM7202...,1,川Z2DG97,前叶子板（右）,1615.00,1606.38,239.40,239.40,喷漆,未减,减损,前叶子板（右）,0,A,0,A
3,RDAA2021510100S0256472,LSGDR53L8MH112602,2021 上汽通用凯迪拉克 凯迪拉克CT4 三厢 2.0T 手自一体 尊贵型(SGM7202...,1,川Z2DG97,前保险杠皮,2394.00,1591.46,239.40,239.40,喷漆,未减,减损,前保险杠皮,0.606905,D,1.162281,E
4,RDAA2021510100S0256472,LSGDR53L8MH112602,2021 上汽通用凯迪拉克 凯迪拉克CT4 三厢 2.0T 手自一体 尊贵型(SGM7202...,1,川Z2DG97,前保险杠下隔栅（右）,80.00,79.57,-1.00,-1.00,无,未减,未减,-1,0,A,,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670316,RDAA2022511600S0000883,LGBF7AE03LR249454,东风日产 天籁 2020 2.0L CVT,1,川XJR258,散热器框架,1425.15,1425.15,181.91,181.91,拆装,未减,减损,散热器框架,0.012532,A,0.540047,D
670317,RDAA2022511600S0000883,LGBF7AE03LR249454,东风日产 天籁 2020 2.0L CVT,1,川XJR258,冷凝器,2062.97,2062.97,138.60,138.60,拆装,未减,减损,冷凝器,0.002632,A,0.289971,C
670318,RDAA2022511600S0000883,LGBF7AE03LR249454,东风日产 天籁 2020 2.0L CVT,1,川XJR258,前保险杠下导流板,578.03,578.03,25.99,25.99,拆装,未减,减损,前保险杠下导流板,0.001782,A,0.044248,A
670319,RDAA2022511600S0000883,LGBF7AE03LR249454,东风日产 天籁 2020 2.0L CVT,1,川XJR258,散热器护板（左）,99.66,99.66,-1.00,-1.00,无,未减,未减,-1,0.003211,A,,-1


In [7]:
dic1 = {}
for index, value in enumerate(set(y['车型名称'])):
    dic1[value]= index

dic2 = {}
for index, value in enumerate(set(y['compname'])):
    dic2[value]= index

dic3 = {}
for index, value in enumerate(set(y['工时名称'])):
    dic3[value]= index

dic4 = {}
for index, value in enumerate(set(y['hourComname'])):
    dic4[value]= index
    
dic5 = {}
for index, value in enumerate(set(y['配件减损'])):
    dic5[value]= index
    
dic6 = {}
for index, value in enumerate(set(y['工时减损'])):
    dic6[value]= index

In [8]:
X = np.vstack((y['车型名称'].map(lambda x: dic1.get(x)), y['修理厂类型'].astype('str'), y['compname'].map(lambda x: dic2.get(x)), 
               y['配件定损'].astype('str'), y['配件核损'].astype('str'), y['工时定损小计'].astype('str'), 
               y['工时核损小计'].astype('str'), y['工时名称'].map(lambda x: dic3.get(x)), y['配件减损'].map(lambda x: dic5.get(x)), 
               y['工时减损'].map(lambda x: dic6.get(x)), y['hourComname'].map(lambda x: dic4.get(x)))).T

In [9]:
X

array([[2857, '1', 24201, ..., 1, 1, 8687],
       [2857, '1', 26674, ..., 1, 0, 7889],
       [2857, '1', 12134, ..., 1, 0, 6689],
       ...,
       [9460, '1', 1651, ..., 1, 0, 2046],
       [9460, '1', 7337, ..., 1, 1, 8687],
       [9460, '1', 11233, ..., 1, 0, 5510]], dtype=object)

In [10]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
multi_target_forest.fit(X, Y).predict(X)

array([['A', '-1'],
       ['A', 'A'],
       ['A', 'A'],
       ...,
       ['A', 'A'],
       ['A', '-1'],
       ['A', 'C']], dtype=object)

In [11]:
result = multi_target_forest.predict(X)

In [12]:
a = pd.DataFrame(result)

In [13]:
y['result1'] = a[[0]]
y['result2'] = a[[1]]

In [15]:
loo1 = y[y['（配）所属区间'].astype('str') != y['result1'].astype('str')]
print(loo1)

                      registno            frameno  \
1106    RDAA2020513400S0018833  LZWADAGA9HE514969   
2702    RDAA2021513000S0042607  LSVXZ25N6E2250422   
46076   RDAA2021510100S0247887  LDCC132T4G1488565   
58207   RDAA2021510100S0365655  LVHFC168XK6073745   
58213   RDAA2021510100S0365655  LVHFC168XK6073745   
68533   RDAA2021510100S0341627  LE4WG7GB7LL616510   
77057   RDAA2021510100S0352800  LBV8V3104GMD52506   
161872  RDZA2021510100S0346351  LFMAXACC2M0273729   
250338  RDAA2021511000S0019147  LBV5S3109FSL12991   
433372  RDAA2021513000S0044162  L6T7822Z0KD006330   
484964  RDAA2021513400S0027659  LSGJM84J1FY141197   
521218  RDAA2021510100S0367089  LGXCE6DB6K0042739   
537824  RDAA2021519400S0043713  LVSHFFAC4KF766510   
616502  RDAA2021513900S0012186  LS4ASE2E8LJ110105   
629463  RDAA2022510100S0004229  LGBL4AE03GD030685   

                                                  车型名称  修理厂类型 licenseno  \
1106         2016 上汽通用五菱 荣光 微客 1.5L 手动档 实用型(LZW6446JY)      1   川WYP152   
2

In [16]:
len(loo1)

15

In [17]:
loo2 = y[y['（工）所属区间'].astype('str') != y['result2'].astype('str')]
print(loo2)

                      registno            frameno  \
31658   RDAA2021510800S0029609  LBECFAHB4FZ196694   
31659   RDAA2021510800S0029609  LBECFAHB4FZ196694   
37687   RDAA2021510100S0340990  LZZ1ELSF6JM309300   
38349   RDAA2021513400S0027047  LVZA53P92JC025997   
38372   RDAA2021513400S0027047  FS512587451256985   
...                        ...                ...   
619167  RDAA2021511300S0048139  L547TU82375982523   
619196  RDAA2021511300S0048139  LJDMAA223J00J4915   
619197  RDAA2021511300S0048139  LJDMAA223J00J4915   
619199  RDAA2021511300S0048139  LJDMAA223J00J4915   
619201  RDAA2021511300S0048139  LJDMAA223J00J4915   

                                                     车型名称  修理厂类型 licenseno  \
31658            2014 北京现代 名图 三厢 1.8L 手自一体 智能型(BH7180PAY)      1   川HS6525   
31659            2014 北京现代 名图 三厢 1.8L 手自一体 智能型(BH7180PAY)      1   川HS6525   
37687                   中国重汽 豪泺(HOWO) 手动档 (ZZ3257N4647E1)      3   川AAL905   
38349        2018 东风 东风小康 SUV 1.8L 手动档 舒适型升级版(DXK64

In [18]:
len(loo2)

91